# FSM

## Imports

In [17]:
import pandas as pd
import numpy as np

In [18]:
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
raw_df = pd.read_csv('../data/archive/1429_1.csv')
    
df = raw_df[['asins', 'reviews.rating', 'reviews.doRecommend', 'reviews.title', 'reviews.text']]
    
df.dropna(inplace=True)

/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [12]:
df.sample(3)

,asins,reviews.rating,reviews.doRecommend,reviews.title,reviews.text
24331,"B00L9EPT8O,B01E6AO69U",4.0,True,Alexa good start to home automation and companion,"Alexa is great. She plays music for me, tells ..."
19035,B00OQVZDJM,5.0,True,My first kindle,I purchased this product a few months ago and ...
31222,B00U3FPN4U,5.0,True,Easy to use,This was a big help in cutting the cable cord ...


In [15]:
text = "Idk what I want to do with this project, I'll make a model or two but I think I want to make this more of an analysis rather than a data science project"

## Cleaning

In [19]:
sw = stopwords.words('english')

In [20]:
#from last NLP project, written with Amanda Potter 
def doc_preparer(doc, stop_words=sw):
    '''
    
    :param doc: a document from the corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in stop_words]
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)

In [21]:
#from last NLP project, written with Amanda Potter 
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
  

def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [23]:
X = df['reviews.text']
y = df['reviews.doRecommend']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12, test_size=.15)
X_t, X_val, y_t, y_val = train_test_split(X_train, y_train, random_state=12, test_size=.20)

In [24]:
b = "why are you doing this to me git why"